In [ ]:
import numpy as np
from matplotlib import colors
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib import rc
import pickle
import signac as sg
from scipy.special import gamma
import copy as copy
import scipy
from global_functions import adjustmaps
import h5py
from scipy.interpolate import make_lsq_spline
from itertools import product

In [ ]:
# Update global plotting parameters
rc('axes', labelsize=16)  # Font size for x and y labels
rc('axes', titlesize=16)
rc('xtick', labelsize=15)  # Font size for x-axis tick labels
rc('ytick', labelsize=15)  # Font size for y-axis tick labels
rc('lines', markersize=12)  # Set the default marker size to 10 points
rc('legend', fontsize=14)

# Define/load things non-specific to a given set of results
metric = "P_s"
Aeff = 1.0
t_final = 600
ncell_tot = 87_993
c = 1.42
with sg.H5Store('shared_data.h5').open(mode='r') as sd:
    b_vec = np.array(sd['b_vec'])
tau_vec = b_vec * gamma(1+1/c)
tau_step = np.diff(tau_vec)[0] / 2
tau_edges = np.concatenate(([0], np.arange(tau_step/2, tau_vec[-1]+tau_step, tau_step)))
tauc_methods = ["flat"]

In [ ]:
def set_globals(results_pre):
    if results_pre == 'meta_mean_results':
        globals()['metric_lab'] = f'$<{metric}>_{{meta}}$'
    else:
        globals()['metric_lab'] = f'$P(<{metric}>_k \geq 0.5)$'
    globals()['fn_prefix'] = f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
    globals()['fig_prefix'] = f"{results_pre}/figs/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"

    # Load things saved specific to these results
    with open(fn_prefix + 'metric_data.pkl', 'rb') as handle:
        globals()['metric_data'] = pickle.load(handle)
    globals()['all_metric'] = metric_data['all_metric']
    globals()['all_tau'] = np.load(f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/all_tau.npy")
    globals()['C_vec'] = np.load(fn_prefix + "C_vec.npy")
    globals()['ncell_vec'] = np.load(fn_prefix + "ncell_vec.npy")
    globals()['slice_left_all'] = np.load(fn_prefix + "slice_left_all.npy")
    eps_axes = {}
    with h5py.File(fn_prefix + "/eps_axes.h5", "r") as handle:
        for key in handle.keys():
            eps_axes.update({key: handle[key][()]})
    globals()['eps_axes'] = eps_axes

In [ ]:
# Define set of results
# results_pre = 'meta_mean_results'
results_pre = 'gte_thresh_results'
if results_pre == 'meta_mean_results':
    metric_lab = f'$<{metric}>_{{meta}}$'
else:
    metric_lab = f'$P(<{metric}>_k \geq 0.5)$'
fn_prefix = f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
fig_prefix = f"{results_pre}/figs/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"

# Load things saved specific to these results
with open(fn_prefix + 'metric_data.pkl', 'rb') as handle:
    metric_data = pickle.load(handle)
all_metric = metric_data['all_metric']
all_tau = np.load(f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/all_tau.npy")
C_vec = np.load(fn_prefix + "C_vec.npy")
ncell_vec = np.load(fn_prefix + "ncell_vec.npy")
slice_left_all = np.load(fn_prefix + "slice_left_all.npy")
eps_axes = {}
with h5py.File(fn_prefix + "/eps_axes.h5", "r") as handle:
    for key in handle.keys():
        eps_axes.update({key: handle[key][()]})

### Preprocess robustness

In [ ]:
# for results_pre in ['meta_mean_results', 'gte_thresh_results']:
for results_pre in ['gte_thresh_results']:
    # Load things saved specific to these results
    set_globals(results_pre)

    # Collect <metric> across all state variables and uncertainty parameterizations
    shape = [len(eps_axes[key]) for key in eps_axes.keys()] 
    shape += [len(C_vec), len(ncell_vec), len(slice_left_all)]
    phase_full = np.ones((shape)) * np.nan
    with h5py.File(fn_prefix + "phase_flat.h5", "r") as phase_handle:
        for eps_params in product(*eps_axes.values()):
            # Create a dictionary of indices along with values for the current combination
            eps_params_dict = {
                key: (index, np.round(value, 3)) for key, values in zip(eps_axes.keys(), eps_axes.values())
                for index, value in enumerate(values) if value == eps_params[list(eps_axes.keys()).index(key)]
            }

            # Get phase slice at this epsilon parameterization
            data_key = f"{eps_params_dict['mu_tau'][1]}/{eps_params_dict['sigm_tau'][1]}/"
            data_key += f"{eps_params_dict['mu_tauc'][1]}/{eps_params_dict['sigm_tauc'][1]}/phase"
            phase_slice = phase_handle[data_key][:]

            # Add them to collective phase_all
            eps_indices = [val[0] for val in eps_params_dict.values()]
            index_tuple = tuple(eps_indices) + (slice(None), slice(None), slice(None))
            phase_full[index_tuple] = phase_slice
    # Save full phase matrix to file
    np.save(fn_prefix + "phase_full.npy", phase_full)

    rob_thresh_vec = np.linspace(min(phase_full.flatten()), max(phase_full.flatten()), 100)
    np.save(fn_prefix + "rob_thresh_vec.npy", rob_thresh_vec)
    allrob = np.ones((rob_thresh_vec.size, C_vec.size, ncell_vec.size, slice_left_all.size)) * np.nan
    maxrob = np.ones((len(rob_thresh_vec), len(C_vec))) * np.nan
    argmaxrob = np.ones((len(rob_thresh_vec), len(C_vec), 2)) * np.nan
    tot_eps_samples = np.cumprod([len(axis) for axis in eps_axes.values()])[-1]
    zero_eps_i = [np.argwhere(ax == 0)[0][0] for ax in eps_axes.values()]
    for (thresh_i, thresh), (C_i, C) in product(enumerate(rob_thresh_vec), 
                                                enumerate(C_vec)):
        rob_slice = np.ones((len(ncell_vec), len(slice_left_all))) * np.nan
        for (ncell_i, ncell), (sl_i, sl) in product(enumerate(ncell_vec),
                                                    enumerate(slice_left_all)):
            # First, check that this result is feasible with zero uncertainty
            # Skip and keep at nan if not feasible
            metric_zero_eps = phase_full[tuple(zero_eps_i + [C_i, ncell_i, sl_i])]
            if np.isnan(metric_zero_eps) or (metric_zero_eps < thresh): continue
            # Now, get the robstness at this (C,ncell,slice_left) coordinate and store
            counts = np.count_nonzero(phase_full[..., C_i, ncell_i, sl_i] >= thresh)
            robustness = counts / tot_eps_samples
            rob_slice[ncell_i, sl_i] = robustness
        allrob[thresh_i, C_i] = rob_slice
        if np.any(~np.isnan(rob_slice)):
            # Store the max robustness at this (thresh, C) coordinate
            maxrob[thresh_i, C_i] = np.nanmax(rob_slice)
            # Also store the optimal param indices
            optimal_param_i = np.unravel_index(np.nanargmax(rob_slice, axis=None), rob_slice.shape)
            argmaxrob[thresh_i, C_i] = optimal_param_i
    # Save maxrob and argmaxrob to files
    np.save(fn_prefix + "maxrob.npy", maxrob)
    np.save(fn_prefix + "argmaxrob.npy", argmaxrob)

### $\text{max}(\omega)$ vs target $<\text{metric}_{meta}>$

In [ ]:
set_globals('gte_thresh_results')
maxrob = np.load(fn_prefix + "maxrob.npy")
argmaxrob = np.load(fn_prefix + "argmaxrob.npy")
rob_thresh_vec = np.load(fn_prefix + "rob_thresh_vec.npy")

cbar_labels = [fr"optimally robust range fraction for intervention",
               fr"optimally robust slice left for intervention"]
for opt_i in range(2):
    fig, ax = plt.subplots(1, figsize=(11,7))
    colormap = copy.copy(cm.RdPu_r)
    if opt_i == 0:
        vmax = 1; vmin = 0
    else:
        # vmax = max(slice_left_all); vmin = min(slice_left_all)
        # vmax = slice_left_all[np.nanmax(argmaxrob[:,9,1]).astype(int)] / ncell_tot; vmin=0
        vmax = 1; vmin = 0
    normalize = colors.Normalize(vmin=vmin, vmax=vmax)

    all_markers = ['o','^','s','D']
    all_linestyles = ['dotted', 'dashdot', 'dashed', 'solid']
    C_i_samples = [0,3,6,9]
    # C_i_samples = [9]
    for line_i, C_i in enumerate(C_i_samples):
        plot_vec = np.ones(len(rob_thresh_vec)) * np.nan
        c_vec = np.ones(len(rob_thresh_vec)) * np.nan
        for thresh_i, thresh in enumerate(rob_thresh_vec):
            if thresh_i % 2 == 0: continue
            # Get the maximum robustness across (ncell, sl) at this C 
            if maxrob[thresh_i, C_i] < 1:
                plot_vec[thresh_i] = maxrob[thresh_i, C_i]
            if not np.isnan(plot_vec[thresh_i]):
                if opt_i == 0:
                    c_vec[thresh_i] = ncell_vec[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
                else:
                    c_vec[thresh_i] = slice_left_all[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
        scatter = ax.scatter(rob_thresh_vec, plot_vec, cmap=colormap, norm=normalize,
                            c=c_vec, marker=all_markers[line_i])#, s=60)
        ax.scatter([], [], label=fr"$C~/~k_{{max}}=${np.round(C_vec[C_i]/ncell_tot, 1)}", 
                   s=60, c='black', marker=all_markers[line_i])
    cbar = fig.colorbar(scatter, ax=ax, location="right", shrink=0.8)
    cbar.set_label(cbar_labels[opt_i], rotation=-90, labelpad=30)
    ax.set_ylabel(fr"$\text{{max}}(\omega)$")
    ax.set_xlabel(fr"target {metric_lab}")
    ax.legend()
    ax.axvline(0.5, ls='--', c='k', lw=5)
    fn = fig_prefix + f"/maxrob_{opt_i}.png"
    fig.savefig(fn, bbox_inches='tight')

In [ ]:
cbar_labels = [fr"optimal range fraction for intervention",
               fr"optimal slice left for intervention"]
for opt_i in range(2):
    fig, ax = plt.subplots(1, figsize=(11,7))
    colormap = copy.copy(cm.RdPu_r)
    if opt_i == 0:
        vmax = 1; vmin = 0
    else:
        # vmax = max(slice_left_all); vmin = min(slice_left_all)
        # vmax = slice_left_all[np.nanmax(argmaxrob[:,9,1]).astype(int)] / ncell_tot; vmin=0
        vmax = 1; vmin = 0
    normalize = colors.Normalize(vmin=vmin, vmax=vmax)

    all_markers = ['o','^','s','D']
    all_linestyles = ['dotted', 'dashdot', 'dashed', 'solid']
    C_i_samples = [0,3,6,9]
    # C_i_samples = [9]
    for line_i, C_i in enumerate(C_i_samples):
        plot_vec = np.ones(len(rob_thresh_vec)) * np.nan
        c_vec = np.ones(len(rob_thresh_vec)) * np.nan
        for thresh_i, thresh in enumerate(rob_thresh_vec):
            if thresh_i % 2 == 0: continue
            # Get the maximum robustness across (ncell, sl) at this C 
            if maxrob[thresh_i, C_i] < 1:
                plot_vec[thresh_i] = maxrob[thresh_i, C_i]
            if not np.isnan(plot_vec[thresh_i]):
                if opt_i == 0:
                    c_vec[thresh_i] = ncell_vec[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
                else:
                    c_vec[thresh_i] = slice_left_all[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
        scatter = ax.scatter(plot_vec, rob_thresh_vec, cmap=colormap, norm=normalize,
                            c=c_vec, marker=all_markers[line_i])#, s=60)
        ax.scatter([], [], label=fr"$C~/~k_{{max}}=${np.round(C_vec[C_i]/ncell_tot, 1)}", 
                   s=60, c='black', marker=all_markers[line_i])
    cbar = fig.colorbar(scatter, ax=ax, location="right", shrink=0.8)
    cbar.set_label(cbar_labels[opt_i], rotation=-90, labelpad=30)
    ax.set_xlabel(fr"$\omega$ requirement")
    ax.set_ylabel(fr"max(target {metric_lab})")
    ax.legend()
    # ax.axvline(0.5, ls='--', c='k', lw=5)
    # fn = fig_prefix + f"/maxrob_{opt_i}.png"
    # fig.savefig(fn, bbox_inches='tight')

### Compare optimal strategies between metapop metrics

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,7))
# colormap = copy.copy(cm.RdPu_r)
# colormap = copy.copy(cm.PuBu_r)
colormap = copy.copy(cm.copper)
vmin = 0; vmax = 1
normalize = colors.Normalize(vmin=vmin, vmax=vmax)
colormap_2 = copy.copy(cm.YlGn_r)
normalize_2 = colors.Normalize(vmin=vmin, vmax=vmax)
C_i = 9
markers = ['o', '^']
res_labs = [f'$<{metric}>_{{meta}}$', f'$P(<{metric}>_k \geq 0.5)$']
# for res_i, results_pre in enumerate(['meta_mean_results']):#, 'gte_thresh_results']):
for res_i, results_pre in enumerate(['meta_mean_results', 'gte_thresh_results']):
    set_globals(results_pre)
    maxrob = np.load(fn_prefix + "maxrob.npy")
    argmaxrob = np.load(fn_prefix + "argmaxrob.npy")
    rob_thresh_vec = np.load(fn_prefix + "rob_thresh_vec.npy")
    
    opt_ncell = np.ones_like(rob_thresh_vec) * np.nan
    opt_sl = np.ones_like(rob_thresh_vec) * np.nan
    for thresh_i, thresh in enumerate(rob_thresh_vec):
        if maxrob[thresh_i, C_i] < 1:
            opt_ncell[thresh_i] = ncell_vec[int(argmaxrob[thresh_i, C_i][0])]
            opt_sl[thresh_i] = slice_left_all[int(argmaxrob[thresh_i, C_i][1])]
    opt_ncell = opt_ncell / ncell_tot
    opt_sl = opt_sl / ncell_tot
    ax.plot(opt_ncell, opt_sl, c='k', zorder=-1)
    # scatter = ax.scatter(opt_ncell, opt_sl, cmap=colormap_2, norm=normalize_2, c=rob_thresh_vec, marker=markers[res_i], s=600)
    scatter = ax.scatter(opt_ncell, opt_sl, cmap=colormap, norm=normalize, c=maxrob[:, C_i], marker=markers[res_i], s=200)
    ax.scatter([], [], c='k', marker=markers[res_i], label=res_labs[res_i])
cbar = fig.colorbar(scatter, ax=ax, location="right", shrink=0.8)
cbar.set_label(r"$\omega$ requirement", rotation=-90, labelpad=30)
ax.set_xlabel(fr"optimal $n_{{cell}}$")
ax.set_ylabel(fr"optimal $sl$")
ax.legend()
# ymin = -0.05; ymax = 1
ymin = min(slice_left_all/ncell_tot) - np.diff(slice_left_all/ncell_tot)[0]
ymax = max(slice_left_all/ncell_tot)
# ymax = 0.6
ax.set_ylim(ymin,ymax)
xmin = min(ncell_vec/ncell_tot)
# xmin = ncell_vec[5]/ncell_tot
xmax = max(ncell_vec/ncell_tot)
ax.set_xlim(xmin, xmax)
grid_style = {
    'color': 'k', 
    'linewidth': 1, 
    'linestyle': '-',
    'alpha': 0.2
}
ax.vlines(ncell_vec/ncell_tot, ymin, ymax, zorder=-1, **grid_style)
ax.hlines(slice_left_all/ncell_tot, xmin, xmax, zorder=-1, **grid_style)
sl_constraint = 1-(ncell_vec/ncell_tot)
# ax.plot(ncell_vec/ncell_tot, sl_constraint)
ax.fill_between(ncell_vec/ncell_tot, np.repeat(1, ncell_vec.size), sl_constraint, where=(1 > sl_constraint), color='gray', alpha=1, label='Shaded Area')

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,7))
# colormap = copy.copy(cm.RdPu_r)
# colormap = copy.copy(cm.PuBu_r)
colormap = copy.copy(cm.copper)
vmin = 0; vmax = 0.6
normalize = colors.Normalize(vmin=vmin, vmax=vmax)
C_i = 9
markers = ['o', '^']
res_labs = [f'$<{metric}>_{{meta}}$', f'$P(<{metric}>_k \geq 0.5)$']
for res_i, results_pre in enumerate(['meta_mean_results', 'gte_thresh_results']):
    set_globals(results_pre)
    maxrob = np.load(fn_prefix + "maxrob.npy")
    argmaxrob = np.load(fn_prefix + "argmaxrob.npy")
    rob_thresh_vec = np.load(fn_prefix + "rob_thresh_vec.npy")
    
    opt_ncell = np.ones_like(rob_thresh_vec) * np.nan
    opt_sl = np.ones_like(rob_thresh_vec) * np.nan
    # max_target= np.ones_like(rob_thresh_vec) * np.nan
    for thresh_i, thresh in enumerate(rob_thresh_vec):
        if maxrob[thresh_i, C_i] < 1:
            # max_target[thresh_i] = maxrob[thresh_i, C_i]
            opt_ncell[thresh_i] = ncell_vec[int(argmaxrob[thresh_i, C_i][0])]
            opt_sl[thresh_i] = slice_left_all[int(argmaxrob[thresh_i, C_i][1])]
    opt_ncell = opt_ncell / ncell_tot
    opt_sl = opt_sl / ncell_tot
    ax.plot(maxrob[:, C_i], opt_ncell, c='k', zorder=-1)
    scatter = ax.scatter(maxrob[:, C_i], opt_ncell, cmap=colormap, norm=normalize, c=opt_sl, marker=markers[res_i])
    ax.scatter([], [], c='k', marker=markers[res_i], label=res_labs[res_i])
    # scatter = ax.scatter(opt_ncell/ncell_tot, opt_sl/ncell_tot, cmap=colormap, norm=normalize, c=maxrob[:, C_i])
cbar = fig.colorbar(scatter, ax=ax, location="right", shrink=0.8)
cbar.set_label(fr"optimal $sl$", rotation=-90, labelpad=30)
ax.set_ylabel(fr"optimal $n_{{cell}}$")
ax.set_xlabel(r"$\omega$ requirement")
ax.legend()

In [ ]:
cbar_labels = [fr"optimal range fraction for intervention",
               fr"optimal slice left for intervention"]
for opt_i in range(2):
    fig, ax = plt.subplots(1, figsize=(11,7))
    colormap = copy.copy(cm.RdPu_r)
    if opt_i == 0:
        vmax = 1; vmin = 0
    else:
        # vmax = max(slice_left_all); vmin = min(slice_left_all)
        # vmax = slice_left_all[np.nanmax(argmaxrob[:,9,1]).astype(int)] / ncell_tot; vmin=0
        vmax = 1; vmin = 0
    normalize = colors.Normalize(vmin=vmin, vmax=vmax)

    all_markers = ['o','^','s','D']
    all_linestyles = ['dotted', 'dashdot', 'dashed', 'solid']
    C_i = 9
    # for line_i, C_i in enumerate(C_i_samples):
    for res_i, results_pre in enumerate(['meta_mean_results', 'gte_thresh_results']):
        set_globals(results_pre)
        maxrob = np.load(fn_prefix + "maxrob.npy")
        argmaxrob = np.load(fn_prefix + "argmaxrob.npy")
        rob_thresh_vec = np.load(fn_prefix + "rob_thresh_vec.npy")
        
        plot_vec = np.ones(len(rob_thresh_vec)) * np.nan
        c_vec = np.ones(len(rob_thresh_vec)) * np.nan
        for thresh_i, thresh in enumerate(rob_thresh_vec):
            if thresh_i % 2 == 0: continue
            # Get the maximum robustness across (ncell, sl) at this C 
            if maxrob[thresh_i, C_i] < 1:
                plot_vec[thresh_i] = maxrob[thresh_i, C_i]
            if not np.isnan(plot_vec[thresh_i]):
                if opt_i == 0:
                    c_vec[thresh_i] = ncell_vec[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
                else:
                    c_vec[thresh_i] = slice_left_all[int(argmaxrob[thresh_i, C_i][opt_i])] / ncell_tot
        scatter = ax.scatter(plot_vec, rob_thresh_vec, cmap=colormap, norm=normalize,
                            c=c_vec, marker=all_markers[res_i])#, s=60)
        ax.scatter([], [], label=fr"{metric_lab}", 
                   s=60, c='black', marker=all_markers[res_i])
    cbar = fig.colorbar(scatter, ax=ax, location="right", shrink=0.8)
    cbar.set_label(cbar_labels[opt_i], rotation=-90, labelpad=30)
    ax.set_xlabel(fr"$\omega$ requirement")
    ax.set_ylabel(fr"max(target meta metric)")
    ax.legend()
    # ax.axvline(0.5, ls='--', c='k', lw=5)
    # fn = fig_prefix + f"/maxrob_{opt_i}.png"
    # fig.savefig(fn, bbox_inches='tight')

### $\epsilon$ correlations

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,6))
C_i = 9
num_eps = len(eps_axes.items())
eps_labels = ["|$\mu_T$|", "$|\mu_{\hat{T}}|$", "$\sigma_T$", "$\sigma_{\hat{T}}$"]
color_vec = ['#66CCEE', '#EE6677', '#4477AA', '#AA3377']
for eps_ax_i, eps_ax in enumerate(eps_axes.values()):
    plot_vec = np.ones(ncell_vec.size) * np.nan
    for ncell_i, ncell in enumerate(ncell_vec):
        all_x = []
        all_y = []
        for eps_val_i, eps_val in enumerate(eps_ax):
            eps_slice = tuple([slice(None) if i != eps_ax_i else eps_val_i for i in range(num_eps)])
            full_slice = eps_slice + (C_i, ncell_i, slice(None))
            phase_slice = phase_full[full_slice]
            flat_nonan = phase_slice.ravel()[~np.isnan(phase_slice.ravel())]
            all_y.append(flat_nonan.tolist())
            all_x.append(np.repeat(np.abs(eps_val), len(flat_nonan)).tolist())
        spear = scipy.stats.spearmanr(a=np.array(all_x).flatten(), b=np.array(all_y).flatten())
        plot_vec[ncell_i] = spear.statistic
    ax.plot(ncell_vec/ncell_tot, plot_vec, label=eps_labels[eps_ax_i], marker='o', color=color_vec[eps_ax_i])
ax.axhline(0, ls='--', c='k')
ax.legend()
ax.set_xlim(0, 1)
ax.set_xlabel(r"fraction of species range where $\tau$ altered")
ax.set_ylabel(rf"{metric_lab} correlation to $\epsilon$ axes")
ax.set_title(fr"$C~/~k_{{max}}=${np.round(C_vec[C_i]/ncell_tot, 1)}")
fn = fig_prefix + f"epscorr.png"
fig.savefig(fn, bbox_inches='tight')

### $\text{max}(<\text{metric}_{meta}>$ under $\epsilon=0$

In [ ]:
with h5py.File(fn_prefix + "/phase_flat.h5", "r") as phase_handle:
    data_key = "0.0/0.0/0.0/0.0/phase"
    phase_slice_zeroeps = phase_handle[data_key][:]
    nochange_zeroeps = phase_handle["0.0/0.0/0.0/0.0/metric_nochange"][()]

In [ ]:
fig, ax = plt.subplots(1, figsize=(11,8))

plot_vec = np.ones_like(C_vec) * np.nan
c_vec = np.ones_like(C_vec) * np.nan
for C_i, C in enumerate(C_vec):
    argmax = np.nanargmax(phase_slice_zeroeps[C_i, :, :])
    optimal_param_i = np.unravel_index(argmax, phase_slice_zeroeps.shape[1:])
    # print(C_i, optimal_param_i)
    plot_vec[C_i] = phase_slice_zeroeps[C_i, optimal_param_i[0], optimal_param_i[1]]
    c_vec[C_i] = ncell_vec[optimal_param_i[0]]
c_vec = c_vec / ncell_tot
vmax = np.nanmax(c_vec); vmin = np.nanmin(c_vec)
vmin = 0; vmax = 1
vmin = min(ncell_vec) / ncell_tot; vmax = max(ncell_vec) / ncell_tot
norm = colors.Normalize(vmin=vmin, vmax=vmax)
colormap = copy.copy(cm.RdPu_r)
sm = cm.ScalarMappable(cmap=colormap, norm=norm)
bar_colors = colormap(norm(c_vec))
bar = ax.bar(np.arange(C_vec.size), plot_vec, color=bar_colors)
cbar = plt.colorbar(sm, label='Color Scale Value', ax=ax)
cbar.set_label(fr"optimal fraction of range for intervention", rotation=-90, labelpad=30)
ax.set_ylim(nochange_zeroeps, 1.05*np.max(plot_vec))
ax.set_ylabel(fr"$\text{{max}}(${metric_lab}$)$")
xtick_spacing = 1
xticks = np.arange(0, len(C_vec), xtick_spacing)
ax.set_xticks(xticks, labels=np.round((C_vec/(ncell_tot))[::xtick_spacing], 1));
ax.set_xlabel(r"shift in $\tau$ per cell if spread across entire range $({C}~/~{k_{max}})$")

fn = fig_prefix + f"maxmetric_C_zeroeps.png"
fig.savefig(fn, bbox_inches='tight')

### Visualize fire regime change

In [ ]:
# Read in maps and convert fdm to tau
ul_coord = [1500, 2800]
lr_coord = [2723, 3905]
usecols = np.arange(ul_coord[0],lr_coord[0])
sdmfn = "../shared_maps/SDM_1995.asc"
sdm = np.loadtxt(sdmfn,skiprows=6+ul_coord[1],
                         max_rows=lr_coord[1], usecols=usecols)
fdmfn = '../shared_maps/FDE_current_allregions.asc'
fdm = np.loadtxt(fdmfn,skiprows=6+ul_coord[1], 
                         max_rows=lr_coord[1], usecols=usecols)
sdm, fdm = adjustmaps([sdm, fdm])
delta_t = 30
b_raster = delta_t / np.power(-np.log(1-fdm), 1/c)
tau_raster = b_raster * gamma(1+1/c)
maps_filt = (sdm > 0) & (fdm > 0)
tau_flat = tau_raster[maps_filt] #+ (max(C_vec)/ncell_tot)
tau_flat[tau_flat > max(tau_vec)] = max(tau_vec)
tau_argsort = np.argsort(tau_flat)

#### Shift examples

In [ ]:
figsize = (9,6)
fig, ax = plt.subplots(1, figsize=figsize)
tau_max = 80
color = 'orangered'
tau_step = np.diff(tau_vec)[0] / 2
tau_edges = np.concatenate(([0], np.arange(tau_step/2, tau_vec[-1]+tau_step, tau_step)))
ax.hist(tau_flat, bins=tau_edges, color=color, histtype='step', lw=3.5); 
ncell = 50_000
# ncell = 19_000
sl = 11_000
tau_slice = tau_flat[tau_argsort][sl:sl+ncell]
ax.hist(tau_slice, bins=tau_edges, color=color, alpha=0.5);
ax.set_yticks([])
ax.set_ylabel(r"$\tau$ prevalence within species range")
ax.set_xlabel(r"$\tau$")
ax.set_xlim(15, tau_max);
# fn = f"figs/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/maxmetric_C_zeroeps.png"
fn = f"{results_pre}/figs/taudist_preshift_{ncell}.png"
fig.savefig(fn, bbox_inches='tight')

fig, ax = plt.subplots(1, figsize=figsize)
tauc = max(C_vec) / ncell
tau_shifted = tau_flat[tau_argsort].copy()
tau_shifted[sl:sl+ncell] += tauc
ax.hist(tau_shifted, bins=tau_edges, color=color, histtype='step', lw=3.5); 
ax.hist(tau_slice+tauc, bins=tau_edges, color=color, alpha=0.5);
ax.set_yticks([])
ax.set_ylabel(r"$\tau$ prevalence within species range")
ax.set_xlabel(r"$\tau$")
ax.set_xlim(15, tau_max);
fn = f"{results_pre}/figs/taudist_postshift_{ncell}.png"
fig.savefig(fn, bbox_inches='tight')

In [ ]:
# Make Bspline interpolation for <metric>
metric_expect_vec = np.ones(tau_vec.size) * np.nan
for tau_i, tau in enumerate(tau_vec):
    tau_filt = (all_tau == tau)
    metric_slice = all_metric[tau_filt]
    metric_expect_vec[tau_i] = np.mean(metric_slice)
t = tau_vec[2:-2:1] 
k = 3
t = np.r_[(0,)*(k+1), t, (tau_vec[-1],)*(k+1)]
spl = make_lsq_spline(tau_vec[1:], metric_expect_vec[1:], t, k)

# Vizualise distribution of $<\text{metric}>$
figsize = (9,6)
fig, ax = plt.subplots(1, figsize=figsize)
color = 'limegreen'
mlab = "$<P_s>_k$"
ax.hist(spl(tau_flat[tau_flat < tau_max]), bins=120, color=color, histtype='step', lw=3.5); 
ax.set_yticks([])
ax.set_ylabel(rf"{mlab} frequency")
ax.set_xlabel(rf"{mlab}")
fn = fig_prefix + f"metricdist_preshift_{ncell}.png"
fig.savefig(fn, bbox_inches='tight')

fig, ax = plt.subplots(1, figsize=figsize)
ax.hist(spl(tau_shifted[tau_shifted < tau_max]), bins=120, color=color, histtype='step', lw=3.5); 
ax.set_yticks([])
ax.set_ylabel(rf"{mlab} frequency")
ax.set_xlabel(rf"{mlab}")
fn = fig_prefix + f"/metricdist_postshift_{ncell}.png"
fig.savefig(fn, bbox_inches='tight')

#### Map change optimizing $\omega$ instead of $\text{metric}_\text{meta}$

In [ ]:
mapindices = np.argwhere(maps_filt)
C_i = 9
num_samples = 5

flat = phase_slice_zeroeps[C_i].ravel()
non_nan_i = np.where(~np.isnan(flat))[0]
flat_sort = non_nan_i[np.argsort(flat[non_nan_i])]
phase_sorted_i = np.unravel_index(flat_sort, phase_slice_zeroeps[C_i].shape)
phase_sorted_i = list((zip(phase_sorted_i[0], phase_sorted_i[1])))
tot_samples = len(phase_sorted_i)
# num_samples = int(tot_samples * 0.025)
print(num_samples)
print(phase_sorted_i[-num_samples:])
tot_tauc_k = np.zeros(maps_filt.shape)
n_k = np.zeros(maps_filt.shape)
tauc_relative = np.zeros(maps_filt.shape)
for i in range(tot_samples - num_samples, tot_samples):
    ncell = ncell_vec[phase_sorted_i[i][0]]
    sl = slice_left_all[phase_sorted_i[i][1]]
    map_slice = mapindices[tau_argsort][sl:sl+ncell]
    for map_i in map_slice:
        tot_tauc_k[map_i[0],map_i[1]] += (C_vec[C_i] / ncell)
        n_k[map_i[0],map_i[1]] += 1
        tauc_relative[map_i[0],map_i[1]] += (C_vec[C_i] / ncell) / tau_raster[map_i[0],map_i[1]]
avg_tauc_k = tot_tauc_k / n_k
tauc_relative = tauc_relative / n_k
np.save(fn_prefix + 'avg_tauc_k.npy', avg_tauc_k)
np.save(fn_prefix + 'tauc_relative.npy', tauc_relative)

target = 0.5
target_i = np.argmin(np.abs(rob_thresh_vec - target))
target = rob_thresh_vec[target_i]
flat = allrob[target_i, C_i].ravel()
non_nan_i = np.where(~np.isnan(flat))[0]
flat_sort = non_nan_i[np.argsort(flat[non_nan_i])]
rob_sorted_i = np.unravel_index(flat_sort, allrob[target_i, C_i].shape)
rob_sorted_i = list((zip(rob_sorted_i[0], rob_sorted_i[1])))
tot_samples = len(rob_sorted_i)
# num_samples = int(tot_samples * 0.05)
print(num_samples)
print(rob_sorted_i[-num_samples:])
tot_tauc_k_rob = np.zeros(maps_filt.shape)
n_k_rob = np.zeros(maps_filt.shape)
tauc_relative_rob = np.zeros(maps_filt.shape)
for i in range(tot_samples - num_samples, tot_samples):
    ncell = ncell_vec[rob_sorted_i[i][0]]
    sl = slice_left_all[rob_sorted_i[i][1]]
    map_slice = mapindices[tau_argsort][sl:sl+ncell]
    for map_i in map_slice:
        tot_tauc_k_rob[map_i[0],map_i[1]] += (C_vec[C_i] / ncell)
        n_k_rob[map_i[0],map_i[1]] += 1
        tauc_relative_rob[map_i[0],map_i[1]] += (C_vec[C_i] / ncell) / tau_raster[map_i[0],map_i[1]]
avg_tauc_k_rob = tot_tauc_k_rob / n_k_rob
tauc_relative_rob = tauc_relative_rob / n_k_rob
np.save(fn_prefix + 'avg_tauc_k_rob.npy', avg_tauc_k_rob)
np.save(fn_prefix + 'tauc_relative_rob.npy', tauc_relative_rob)

In [ ]:
[(7, 17), (8, 14), (8, 15), (8, 17), (8, 16)]

In [ ]:
figsize = (9,6)
fig, ax = plt.subplots(1, figsize=figsize)
tau_max = 80

ax.hist(tau_flat, bins=tau_edges, color='orangered', histtype='step', lw=3.5); 

ncell_i, sl_i = phase_sorted_i[-1]
ncell = ncell_vec[ncell_i]
sl = slice_left_all[sl_i]
tau_slice = tau_flat[tau_argsort][sl:sl+ncell]
ax.hist(tau_slice, bins=tau_edges, color='red', alpha=0.5,
        label=rf'optimizing {metric_lab}');

ncell_i, sl_i = rob_sorted_i[-1]
ncell = ncell_vec[ncell_i]
sl = slice_left_all[sl_i]
tau_slice = tau_flat[tau_argsort][sl:sl+ncell]
ax.hist(tau_slice, bins=tau_edges, color='blue', alpha=0.5, 
        label=rf'optimizing $\omega$ at {metric_lab}$=${np.round(target, 2)}');

ax.set_yticks([])
ax.set_ylabel(r"$\tau$ prevalence within species range")
ax.set_xlabel(r"$\tau$")
ax.set_xlim(15, tau_max);
ax.legend()
fn = fig_prefix + f"optimization_diff.png"
fig.savefig(fn, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

# First replace nans with 0 so we can subtract
imshow_mat = np.where(np.isnan(tauc_relative_rob), 0, tauc_relative_rob) - np.where(np.isnan(tauc_relative), 0, tauc_relative)
# Now make pixels not in either map nan
nan_pixel_filt = np.isnan(tauc_relative_rob) & np.isnan(tauc_relative)
imshow_mat[nan_pixel_filt] = np.nan
# Multiply by 100 to get % change
imshow_mat = imshow_mat * 100
# Crop out border where all nans
nonzero_indices = np.nonzero(~np.isnan(imshow_mat))
row_min, row_max = nonzero_indices[0].min(), nonzero_indices[0].max()
col_min, col_max = nonzero_indices[1].min(), nonzero_indices[1].max()
imshow_mat = imshow_mat[row_min:row_max + 1, col_min:col_max + 1]

vmin = np.nanmin(imshow_mat); vmax = np.nanmax(imshow_mat)
vlim = max([np.abs(extrema) for extrema in [vmin, vmax]])
center = 0
cmap = cm.PuOr.copy()
# cmap = cm.PiYG.copy()
norm = colors.TwoSlopeNorm(vmin=-vlim, vcenter=center, vmax=vlim)
cmap.set_bad(color='black', alpha=0.3)
imshow = ax.imshow(imshow_mat, cmap=cmap, norm=norm)
cbar = fig.colorbar(imshow, ax=ax, location="right", shrink=0.8)
ax.set_xlabel(fr"% change in $\tau$ optimizing robustness instead of stability")
ax.set_xticks([])
ax.set_yticks([])
fn = fig_prefix + f"map_example_1.png"
fig.savefig(fn, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

# First replace nans with 0 so we can subtract
imshow_mat = np.where(np.isnan(avg_tauc_k_rob), 0, avg_tauc_k_rob) - np.where(np.isnan(avg_tauc_k), 0, avg_tauc_k)
# Now make pixels not in either map nan
nan_pixel_filt = np.isnan(avg_tauc_k_rob) & np.isnan(avg_tauc_k)
imshow_mat[nan_pixel_filt] = np.nan
# Crop out border where all nans
nonzero_indices = np.nonzero(~np.isnan(imshow_mat))
row_min, row_max = nonzero_indices[0].min(), nonzero_indices[0].max()
col_min, col_max = nonzero_indices[1].min(), nonzero_indices[1].max()
imshow_mat = imshow_mat[row_min:row_max + 1, col_min:col_max + 1]

vmin = np.nanmin(imshow_mat); vmax = np.nanmax(imshow_mat)
vlim = max([np.abs(extrema) for extrema in [vmin, vmax]])
center = 0
cmap = cm.PuOr.copy()
# cmap = cm.PiYG.copy()
norm = colors.TwoSlopeNorm(vmin=-vlim, vcenter=center, vmax=vlim)
cmap.set_bad(color='black', alpha=0.3)
imshow = ax.imshow(imshow_mat, cmap=cmap, norm=norm)
cbar = fig.colorbar(imshow, ax=ax, location="right", shrink=0.8)
ax.set_xlabel(fr"Change in $\hat{{\tau}}$ optimizing robustness instead of stability")
ax.set_xticks([])
ax.set_yticks([])
fn = fig_prefix + f"map_example.png"
fig.savefig(fn, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
imshow_mat = np.where(maps_filt, tau_raster, np.nan)
# ax.imshow(imshow_mat, norm=colors.LogNorm(vmin=np.nanmin(imshow_mat), vmax=np.nanmax(imshow_mat)-160))
ax.imshow(np.where(maps_filt, fdm, np.nan))

In [ ]:
plt.hist(np.where(maps_filt, fdm, np.nan).ravel(), bins=30);

### Map change in optimal strategies between results

In [ ]:
fn = f"meta_mean_results/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/avg_tauc_k_rob.npy"
avg_tauc_k_rob_mmean = np.load(fn)
fn = f"gte_thresh_results/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/avg_tauc_k_rob.npy"
avg_tauc_k_rob_gtet = np.load(fn)

fig, ax = plt.subplots(1, figsize=(10,10))

# First replace nans with 0 so we can subtract
imshow_mat = np.where(np.isnan(avg_tauc_k_rob_gtet), 0, avg_tauc_k_rob_gtet) - np.where(np.isnan(avg_tauc_k_rob_mmean), 0, avg_tauc_k_rob_mmean)
# Now make pixels not in either map nan
nan_pixel_filt = np.isnan(avg_tauc_k_rob_gtet) & np.isnan(avg_tauc_k_rob_mmean)
imshow_mat[nan_pixel_filt] = np.nan
# Crop out border where all nans
nonzero_indices = np.nonzero(~np.isnan(imshow_mat))
row_min, row_max = nonzero_indices[0].min(), nonzero_indices[0].max()
col_min, col_max = nonzero_indices[1].min(), nonzero_indices[1].max()
imshow_mat = imshow_mat[row_min:row_max + 1, col_min:col_max + 1]

vmin = np.nanmin(imshow_mat); vmax = np.nanmax(imshow_mat)
center = 0
cmap = cm.PuOr.copy()
# cmap = cm.PiYG.copy()
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=center, vmax=vmax)
cmap.set_bad(color='black', alpha=0.3)
imshow = ax.imshow(imshow_mat, cmap=cmap, norm=norm)
cbar = fig.colorbar(imshow, ax=ax, location="right", shrink=0.8)
ax.set_xlabel(fr"Change in $\hat{{\tau}}$ gte thresh minus mean")
ax.set_xticks([])
ax.set_yticks([])
# fn = fig_prefix + f"map_example.png"
# fig.savefig(fn, bbox_inches='tight')

### Visualize $P(\text{metric}|\tau)$

In [ ]:
metric_edges = np.linspace(0, 1, 50)
metric_hist = np.histogram2d(all_tau, all_metric, bins=[tau_edges[::2], metric_edges], density=False)
fig, ax = plt.subplots(figsize=(10,8))
cmap = copy.copy(cm.YlGn)
cmap.set_bad(cmap(0.0))
imshow_mat = np.flip(np.transpose(metric_hist[0]), axis=0)
samples_per_tau = int(len(all_metric)/len(b_vec))
imshow_mat = imshow_mat / samples_per_tau
smoothed = scipy.ndimage.gaussian_filter(imshow_mat, sigma=0.125)
im = ax.imshow(imshow_mat, cmap=cmap,
              norm=colors.LogNorm(vmax=1), interpolation="nearest")
cbar = ax.figure.colorbar(im, ax=ax, location="right", shrink=0.6)
cbar.ax.set_ylabel(rf'Frequency of $P_s$ given ${{\tau}}$', 
                   rotation=-90, labelpad=20)
ytick_spacing = 10
ytick_labels = np.flip(metric_hist[2])[::ytick_spacing]
yticks = np.arange(0,len(metric_hist[2]),ytick_spacing)
ax.set_yticks(yticks, labels=np.round(ytick_labels, decimals=1));
ax.set_ylabel(rf'$P_s$')
xtick_spacing = 10
xtick_labels = np.round(metric_hist[1][::xtick_spacing],1)
xticks = np.arange(0,len(metric_hist[1]),xtick_spacing)
ax.set_xticks(xticks, labels=xtick_labels);
ax.set_xlabel(r'$\tau$')
ax.set_xlim(1, imshow_mat.shape[1]-1)
fn = fig_prefix + f"Pmetric_given_tau.png"
fig.savefig(fn, bbox_inches='tight')